# Asignment 2

### Imports 1.0

## Task 1: Exploratory Data Analysis (EDA)

## Task 2: Data Splitting

## Task 3: Feature Engineering

## Task 4: Baseline Model – Linear Regression

## Task 5: Random Forest Regressor - Model Specification and Training

## Task 6: Gradient Boosting Regressor - Model Specification and Training

## Task 7: Hyperparameter Tuning

## Task 8: Iterative Evaluation and Refinement

## Task 9: Final Model Selection and Testing